In [1]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean

from tf_lab.fundamentals.utils import set_visible_GPUs, reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

import tf_lab.autopredictors.scripts.virt_scan_data as vscan

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch,\
                                           plot_3d_point_cloud_paper

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats, \
                                           paper_pc_completion_experiment_id_best_epoch
                                           
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud



/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
GPU = 2
loss = 'chamfer'

class_type = raw_input('Give me the class type.\n').lower()
class_syn_id = shape_net_category_to_synth_id()[class_type]

incomplete_n_samples = 2048
complete_n_samples = 4096
do_training = False
load_training_data = False
load_model_conf = True
do_evaluation = True
do_detailed_plotting = True

experiment_id, best_epoch = paper_pc_completion_experiment_id_best_epoch(class_type, loss)


experiment_name = str(experiment_id) + '_' + class_type + '_' + str(incomplete_n_samples) + '_' \
                  + str(complete_n_samples) + 'pts_' + loss


top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
complete_pclouds_path = osp.join(top_data_dir, 'ShapeNetPointClouds/from_manifold_meshes/not_scaled', \
                             str(complete_n_samples), class_syn_id)
    
n_input = [incomplete_n_samples, 3]
n_output = [complete_n_samples, 3]

train_dir = osp.join(top_data_dir, 'OUT/models/incomplete_pclouds/paper_vanilla_vscan')
train_dir = osp.join(train_dir, experiment_name)
create_dir(train_dir)

if loss == 'emd':
    max_training_epochs = 100
elif loss == 'chamfer':
    max_training_epochs = 100
    
max_evaluation_epochs = max_training_epochs

Give me the class type.
airplane


In [4]:
seed = 42
np.random.seed(seed)
tf.set_random_seed(seed)

In [5]:
cmpl_file_names = pio.load_filenames_of_input_data(complete_pclouds_path, points_extension)
cmpl_pclouds, cmpl_model_names, syn_ids = pio.load_crude_point_clouds(file_names=cmpl_file_names, n_threads=40)
assert(len(np.unique(syn_ids)) == 1)
print '%d files containing complete point clouds were found.' % (len(cmpl_pclouds), )

4045 files containing complete point clouds were found.


In [6]:
v_scan_top_dir = osp.join(top_data_dir, 'From_Matthias/shapenet_dim32_sdf_pc', class_syn_id)

if do_training or load_training_data:
    train_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/train_models_with_scan_id.txt')
    
    train_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, train_id_file, class_syn_id,
                                                      cmpl_pclouds, cmpl_model_names, n_threads=40,
                                                      n_samples=incomplete_n_samples)
    
    train_data, val_data = vscan.make_validation_from_train_data(train_data, 0.10)
    
if do_evaluation:
    test_id_file = osp.join(top_data_dir, 'From_Matthias/train_test_models/test_models_with_scan_id.txt')
    test_data = vscan.load_single_class_incomplete_dataset(v_scan_top_dir, test_id_file, class_syn_id,
                                                      cmpl_pclouds, cmpl_model_names, n_threads=40,
                                                      n_samples=incomplete_n_samples)
    
n_test_models = len(set([t.split('.')[1] for t in test_data.labels]))
print 'Number of test models =', str(n_test_models)

4470 incomplete point clouds were loaded.
Number of test models = 745


In [8]:
pp = test_data.next_batch(1)
pinc = pp[2].reshape(incomplete_n_samples, 3)
pcom = pp[0].reshape(complete_n_samples, 3)

score1 = accuracy_of_completion(pinc, pcom, 0.02, ret_dists=False)
print score1
score2, c2 = coverage_of_completion(pcom, pinc, 0.02, ret_dists=True)

plot_3d_point_cloud_paper(Point_Cloud(points=pinc));
plot_3d_point_cloud_paper(Point_Cloud(points=pcom), c=c2);

In [9]:
if load_model_conf:
    conf = Conf.load(osp.join(train_dir, 'configuration'))
    conf.consistent_io = None
    conf.encoder_args = {'dropout_prob': 1}
    print conf
else:
    decoder_args = {'layer_sizes': [1024, np.prod(n_output)],
                    'non_linearity': tf.nn.relu
                   }

    encoder_args = {'dropout_prob': 0.5}
    
    conf = Conf(
                n_input = n_input,
                n_output = n_output,
                denoising = True,
                training_epochs = max_training_epochs,
                batch_size = 50,
                loss = loss,
                train_dir = train_dir,
                loss_display_step = 1,
                saver_step = 2,
                learning_rate = 0.0005,
                encoder = enc_dec.encoder_with_convs_and_symmetry,
                encoder_args = encoder_args,
                decoder = enc_dec.decoder_with_fc_only,
                decoder_args = decoder_args
               )
    
    conf.consistent_io = False
    conf.experiment_name = experiment_name
    conf.save(osp.join(conf.train_dir, 'configuration'))
    
reset_tf_graph()
set_visible_GPUs([GPU])
ae = PointNetAutoEncoder(experiment_name, conf)

                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'non_linearity': <function relu at 0x7f7886140758>, 'layer_sizes': [1024, 12288]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'dropout_prob': 1}
               experiment_name: 1_airplane_2048_4096pts_chamfer
                 gauss_augment: None
                  is_denoising: True
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [4096, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 2
                     train_dir: /orions4-zfs/projects/lins2/Panos_Space/DATA/OUT/models/incomplete_pclouds/paper_vanilla_vscan/

ResourceExhaustedError: OOM when allocating tensor with shape[1024,12288]
	 [[Node: 1_airplane_2048_4096pts_chamfer_1/1_airplane_2048_4096pts_chamfer/decoder_fc_1/W/Adam_1/Assign = Assign[T=DT_FLOAT, _class=["loc:@1_airplane_2048_4096pts_chamfer/decoder_fc_1/W"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/gpu:0"](1_airplane_2048_4096pts_chamfer_1/1_airplane_2048_4096pts_chamfer/decoder_fc_1/W/Adam_1, 1_airplane_2048_4096pts_chamfer_1/zeros_33)]]

Caused by op u'1_airplane_2048_4096pts_chamfer_1/1_airplane_2048_4096pts_chamfer/decoder_fc_1/W/Adam_1/Assign', defined at:
  File "/usr/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/orions4-zfs/projects/lins2/.local/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/orions4-zfs/projects/lins2/.local

In [ ]:
if do_training:
    training_stats = []
    training_stats.append(ae.train(train_data, conf))    
    with open(osp.join(conf.train_dir, 'train_stats.txt'), 'a') as fout:
        np.savetxt(fout, np.array(training_stats)[0])

In [23]:
if do_evaluation:    
    # Pick the epoch that minimizes the loss on the validation dataset.
#     saved_epochs = np.array(read_saved_epochs(conf.train_dir))
#     allowable_epochs = saved_epochs[saved_epochs <= max_evaluation_epochs]
#     val_stats = eval_model(ae, conf, val_data, epochs=allowable_epochs, verbose=True)
#     val_loss = np.min(val_stats[:,1])
#     best_epoch = int(val_stats[np.argmin(val_stats[:,1]), 0])
    print 'Best epoch = %d.' % (best_epoch,)         
    ae.restore_model(conf.train_dir, best_epoch)
    top_save_dir = osp.join(conf.train_dir, 'output', 'epoch_' + str(best_epoch))
    test_recon, test_loss, test_feed, test_ids, test_gt = ae.evaluate(test_data, conf)
    save_dir = osp.join(top_save_dir, 'test_predictions')
#     save_reconstructions(save_dir, test_recon, test_gt, test_feed, test_ids) # save ply files of test data.    
#     train_loss = ae.evaluate(train_data, conf)[1]
    
    # Report Accuracy and Coverage of test data.
    n_examples = len(test_recon)
    pred_scores = np.zeros((n_examples, 2))
    for i in xrange(n_examples):
        gt = test_gt[i]
        pred = test_recon[i] 
        pred_scores[i, 0] = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=False)
        pred_scores[i, 1] = coverage_of_completion(gt, pred, thres=0.02, ret_dists=False)
    
            
    print 'Test Median-Accuracy-Coverage:', np.median(pred_scores[:, 0]), np.median(pred_scores[:, 1])
    save_pc_prediction_stats(osp.join(top_save_dir, 'detailed_stats.txt'), test_ids, pred_scores)
    
    with open(osp.join(top_save_dir, 'stats.txt'), 'a') as fout:
#         fout.write('Best Validation Epoch = %d\n' % (best_epoch))
#         fout.write('Validation loss = %f\n' % (val_loss))
#         fout.write('Train loss = %f\n' % (train_loss))
#         fout.write('Test loss = %f\n' % (test_loss))
#         fout.write('Gen. Error (abs, per) = %f %f\n' % (abs(test_loss-train_loss),  abs(test_loss-train_loss) / train_loss ))
#         fout.write('Test Median-Accuracy-Coverage = %f %f\n' % (np.median(pred_scores[:, 0]), np.median(pred_scores[:, 1])))
        fout.write('Test Median Harmonic Mean = %f\n' % (np.median(hmean(pred_scores, 1))))
        fout.write('Number of distinct Test Models = %d (test_files=%d)\n' % (n_test_models, test_data.num_examples))    

Test Median-Accuracy-Coverage: 0.89697265625 0.774169921875


In [ ]:
n_best = 500
n_worst = 50

if do_detailed_plotting:
    plt.ioff()
    h_measure = hmean(pred_scores, axis=1)  
    sort_ind = np.argsort(h_measure)
    worst_n = sort_ind[:n_worst]
    best_n = sort_ind[-n_best:]    
    
    save_dir = osp.join(top_save_dir, str(n_best) +'_best_test_predictions')
    create_dir(save_dir)
    
    for i in xrange(n_best):
        good = best_n[i]
        gt = test_gt[good]
        pred = test_recon[good]   
        feed = test_feed[good]
        model_name = test_ids[good]

        ac_meas, ac_col = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=True)
        cov_meas, cov_col = coverage_of_completion(gt, pred, thres=0.02, ret_dists=True)
                    
        ac_col[ac_col <= 0.02] = 0.021
        cov_col[cov_col <= 0.02] = 0.021

        feed_fig = plot_3d_point_cloud_paper(Point_Cloud(points=feed), in_u_sphere=True, show=False);        
        feed_fig.savefig(osp.join(save_dir, str(i) + '.' + model_name + '.feed.png')); plt.close()

        c = ac_col
        pred_fig = plot_3d_point_cloud_paper(Point_Cloud(points=pred), c=c, in_u_sphere=True, show=False);
        pred_fig.savefig(osp.join(save_dir, str(i) + '.' + model_name + '.prediction.png')); plt.close()

        c = cov_col
        gt_fig = plot_3d_point_cloud_paper(Point_Cloud(points=gt), c=c, in_u_sphere=True, show=False);
        gt_fig.savefig(osp.join(save_dir, str(i) + '.' + model_name + '.gt.png')); plt.close()
        
    for i in xrange(n_worst):
        bad = worst_n[i]
        gt = test_gt[bad]
        pred = test_recon[bad]   
        feed = test_feed[bad]
        model_name = test_ids[bad]

        ac_meas, ac_col = accuracy_of_completion(pred, gt, thres=0.02, ret_dists=True)
        cov_meas, cov_col = coverage_of_completion(gt, pred, thres=0.02, ret_dists=True)
                    
        ac_col[ac_col <= 0.02] = 0.021
        cov_col[cov_col <= 0.02] = 0.021

        feed_fig = plot_3d_point_cloud_paper(Point_Cloud(points=feed), in_u_sphere=True, show=False);        
        feed_fig.savefig(osp.join(save_dir, str(i) + '.' + model_name + '.feed.png')); plt.close()

        c = ac_col
        pred_fig = plot_3d_point_cloud_paper(Point_Cloud(points=pred), c=c, in_u_sphere=True, show=False);
        pred_fig.savefig(osp.join(save_dir, str(i) + '.' + model_name + '.prediction.png')); plt.close()

        c = cov_col
        gt_fig = plot_3d_point_cloud_paper(Point_Cloud(points=gt), c=c, in_u_sphere=True, show=False);
        gt_fig.savefig(osp.join(save_dir, str(i) + '.' + model_name + '.gt.png')); plt.close()